In [2]:
# 1. For reading:

# a WFDBrecord object with signals == None.  

item1 = rdheader('headername') # Function belongs to the module

# a WFDBrecord object with signals != None unless it is a 0 channel file. 
item1 = rdsamp('recordname') # Function belongs to the module. 

# Module function
def rdsamp(recordname):
    item1 = rdheader(recordname)
    item1.signals = rddats(item1.filenames)
    return item1


# Module function
def rdheader(recordname):
    f=open()
    return WFDBrecord

# Module function
def rddats(filenames):
    signals = np.fromfile(r)
    return signals



# 2. For writing

# Should wrhea be a module function or an object method? Can it only target WFDBrecord objects?
# Yes, we don't want it to be able to target strings. That's just a bother, why not just use write()?
# In fact, we can actually make some convenient string list to file functions part of the module. 

# lines is just a list of strings
def lines2header(headername, lines):
    f = open(headername+'.hea','w')
    for l in lines:
        f.write("%s\n" % l)
    f.close()

    

In [1]:
# Write a wfdb record's header and dat files. 0 signal records are allowed.  
# Method belongs to the WFDBrecord class. Because it ONLY works on the class. 
# Usesignals = 0 (none), 1 (physical), 2 (digital) - The signals to use to write the file. 
def wrsamp(self, recordname, usesignals=0): # This belongs in signals.py
    
    # Perform field validity and cohesion checks, and write the header file
    self.wrheader(recordname, usesignals)
    
    # Perform signal cohesiveness checks, and write all the dat files associated with the record
    self.wrdats()
    
    # This could be better ... what if they didn't set nsig? 
    if usesignals == 0:
        if (self.nsig!= None and self.nsig > 0) or (self.p_signals != None) or (self.d_signals != None):
            print("What are you doing?")
            sys.exit("Must specify which signals to use to write the WFDB files."
                     "\nusesignals = 1 for physical (p_signals), or 2 for digital (d_signals).")
    
    # Perform field validity and cohesion checks, and write the header file
    self.wrheader(recordname, usesignals)
    
    # Perform signal cohesiveness checks, and write all the dat files associated with the record
    self.wrdats()

# Write a header file. Method belongs to the WFDBrecord class. 
# Usesignals = 0 (none), 1 (physical), 2 (digital) - The signals to use to write the file. 
def wrheader(self, recordname, usesignals=0):
    
    # Check that signals exist to be used to write the header
    if usesignals == 1:
        if self.p_signals == None:
            sys.exit('usesignals==1 specified to use physical signal, but p_signals field is not defined')
    elif usesignals == 2:
        if self.d_signals == None:
            sys.exit('usesignals==2 specified to use digital signal, but d_signals field is not defined')
    
     # Get all the fields needed to write the header
    writefields = self.getwritefields(usesignals)
    
    # Use the chosen signals fields to potentially set missing required derived fields
    self.setsignalfeatures(usesignals, writefields)
        
    # Check the validity of individual fields used to write the header 
    # Set default values for missing fields if needed, if the field has a default. 
    for f in writefields:
        if self.f == None:
            self.setdefault(f)
        self.checkfield(f) 
    
    # Check the compatibility of fields used to write the header
    self.checkfieldcohesion(usesignals)
    
    # Write the header file
    self.wrheaderfile(recordname)

# Convert the fields into a list of strings, then write those to a file. Method belongs to WFDB class.     
def wrheaderfile(self, recordname)
   
    recordline = []
    signallines = []
    commentlines = []
    line2header(recordname, recordline+signallines+commentlines)
    
# Write a list of strings to a file. Method belongs to module!!! 
def lines2header(recordname, lines):
    f = open(recordname+'.hea','w')
    for l in lines:
        f.write("%s\n" % l)
    f.close()
    
# Write all the dat files for a record. 
# Method belongs to the WFDBrecord class. NOT a function of the module. Does check the signal.  
def wrdats(self): # Don't need record names. The object fields specify. 
    
    # Perform extra check of the signal itself in terms of its values for writing. 
    # Get the final digital values to write to the file. 
    digitalwritesigs = self.checkwritesignals()
    
    # Get a list of the dat files to be written and their formats
    datfiles, datchannels = orderedsetlist(self.filename)
    formats = orderedsetlist(self.fmt)[0]
    
    for i in range(0, len(datfiles)):
        # To do: Consider multi samples per frame writing. 
        wrdatfile(datfiles[i], formats[i], self.signals[:,datchannels[i][0]:(datchannels[i][-1]+1)])
    
# Write a dat file. Signals is digital. 
# Belongs to the module
def wrdatfile(filename, fmt, signals):
    f=open(filename,'wb')
    np.tofile(f, signals)
    

# Returns the unique elements in a list in the order that they appear. 
# Also returns the indices of the original list that correspond to each output element. 
# Used for listing dat files and fmts to write. 
# Belongs to module
def orderedsetlist(fulllist):
    uniquelist = []
    ori = []
    for i in fulllist:
        if i not in uniquelist:
            uniquelist.append(i)
    return uniquelist, channels

# Returns elements in a list without consecutive repeated values. Used to check dat files to write. 
# Belongs to module
def orderednoconseclist(fulllist):
    noconseclist = [fulllist[0]]
    if len(fulllist) == 1:
        return noconseclist
    for i in fulllist:
        if i!= noconseclist[-1]:
            noconseclist.append(i)
    return noconseclist


# Get the list of required fields needed to write a wfdb header file (including multi-segment). 
# Returns the default required fields, the user defined fields, and their dependencies.
# This does NOT return p/d_signals or segments.
# Method of WFDBbaserecord
def getwritefields(self, usesignals=0):
    
    # Single-segment record
    if type(self) == WFDBrecord:
        
        # Get required record specification fields
        writefields=self.getwritesubset(reversed(list(recfieldspecs_s.items())))
        
        # Find out if we need to include signal specification fields. Yes if the number of signals >0. 
        # Depending on 'usesignals', we will have to check different fields to find out. 
        if usesignals== 0:
            errormsg = checkfield(self, 'nsig')
            if errormsg:
                sys.exit(errormsg)
            nsig = self.nsig
        elif usesignals == 1:
            errormsg = checkfield(self, 'p_signals')
            if errormsg:
                sys.exit(errormsg)
            nsig = self.p_signals.shape[1]
        else:
            errormsg = checkfield(self, 'd_signals')
            if errormsg:
                sys.exit(errormsg)
            nsig = self.d_signals.shape[1]
            
        # If nsig>0, get required signal specification fields. DON'T include signals here. It is checked in wrsamp.    
        if nsig>0:
            writefields=writefields+self.getwritesubset(reversed(list(sigfieldspecs.items())))

    # Multi-segment record
    else:
        # Get required record specification fields
        writefields=self.getwritesubset(reversed(list(recfieldspecs_s.items())))
        
        # Get required segment specification fields. DON'T add segments here. it is checked in wrmultiheader.
        writefields=writefields+['seglen', 'segname'] 
    
    # Comments
    if self.comments != None:
        writefields.append('comments')
        
    return writefields


# Helper to getwritefields. 
# Method of WFDBbaserecord
def getwritesubset(self, fieldspecs):
    reqfields=[]
    for f in fieldspecs:
        if f in reqfields:
            continue
        # If the field is default required or has been defined by the user...
        if f[1].write_req or self.f!=None:
            rf=f
            # Add the field and its recurrent dependencies
            while rf!=None:
                reqfields.append(rf)
                rf=fieldspecs[rf].dependency
    return reqfields    


# Use the chosen signals fields to potentially set missing required derived fields
# This method WILL overwrite user input fields. If they did not want that to happen, they would set usesignals = 0
# Method of WFDBrecord
def setsignalfeatures(self, usesignals, writefields):
    
    # Physical signal
    if usesignals == 1:
        
        # First, check the signal
        errormsg = self.checkfield(self, p_signals) # remember this doesn't check against other fields
        if errormsg:
            sys.exit(errormsg)
            
        # Fields potentially set via p_signals: nsig, siglen, d_signals, fmt, gain, baseline, initvalue, checksum
        
        self.nsig = p_signals.shape[1]
        self.siglen = p_signals.shape[0]
        
        # fmt is necessary to obtain gain and baseline.
        if self.fmt == 'None':
            res = estres(self.signals)
            self.fmt = self.nsig*[wfdbfmt(max(res))]
        else:
            errormsg = self.checkfield('fmt', 0) 
            if errormsg:
                sys.exit(errormsg)
        
        # Check fmt before performing adc. No need to check gain and baseline which are automatically calculated.
        self.checkfield('fmt', 0)
        
        # Do ADC and store value in d_signals. 
        print('Calculating optimal gain and baseline values to convert physical signal...')
        self.gain, self.baseline = adcparams(self.p_signals, self.fmt)
        print('Performing ADC and storing result in d_signals field...')
        self.d_signals = self.adc()
        
        if 'initvalue' in writefields:
            self.initvalue = list(self.d_signals[0,:])
        if 'checksum' in writefields:
            self.checksum = calc_checksum(self.d_signals)

    # Digital signal
    elif usesignals == 2:

        # First, check the signal
        errormsg = self.checkfield(self, d_signals) 
        if errormsg:
            sys.exit(errormsg)
            
        # Fields potentially set via d_signals: nsig, siglen, initvalue, checksum

        self.nsig = d_signals.shape[1]
        self.siglen = d_signals.shape[0]
        if 'initvalue' in writefields:
            self.initvalue = list(self.d_signals[0,:])
        if 'checksum' in writefields:
            self.checksum = calc_checksum(self.d_signals) 
    
    # No need to call checkfieldcohesion here. It will be called at the end of wrheader. 
    
    return
                
    


# Check a signal, segment, or comment field to make sure it is a list of appropriate length. 
# Scalars will be extended into lists. 
# Belongs to WFDBbaserecord class
def checkfieldlength(self, field):
    
    if field == 'comments'
        # Single element comment. Convert it into a list.
        if type(getattr(self,field))!= list:
            self.field = [self.field]
    else:
        if field in sigfieldspecs:
            desiredlength = self.nsig
        elif field in segfieldspecs:
            desiredlength = self.nseg
            
        # Extend scalars into lists
        if type(getattr(self,field)) != list:
            self.field = [self.field]*desiredlength

        if type(getattr(self,field))!= list or len(getattr(self,field))!=desiredlength:
            sys.exit('Field: '+field+' must be a list of length '+str(desiredlength))
        

# Check the data type of the specified field.
# Belongs to WFDBbaserecord class
def checkfieldtype(self, field):
    
    # signal, segment, and comment specification fields are lists. Check their elements.    
    if field in signalspecs or field in recfieldspecs_m:
        listcheck = 0
    else:
        listcheck = 1
    
    # signals
    if field in signalspecs:
        allowedtypes = signalspecs_m[field].allowedtypes
    # segments. 
    #elif field in segmentspecs:
        # Fill this in later
    # record specification field
    elif field in recfieldspecs_m:
        allowedtypes = recspecs_m[field].allowedtypes
    # signal specification field        
    elif field in sigfieldspecs:
        allowedtypes = sigfieldspecs[field].allowedtypes
    # segment specification field
    elif field in segfieldspecs:
        allowedtypes = segfieldspecs[field].allowedtypes
    # comment field
    elif field in comfieldspecs:
        allowedtypes = comfieldspecs[field].allowedtypes
                
    # Checking each element in the list
    if listcheck:
        # the 'checkfieldlength' method should have already ensured that the field is a list of appropriate length.
        
        if type(getattr(self,field)) not in allowedtypes:
            print('Error - Field: '+field+' must be one of the following types:', allowedtypes)
            sys.exit()
    # Just checking the single element   
    else:
        for f in getattr(self,field):
            if type(f) not in allowedtypes:
                print('Error - Each element in field: '+field+' must be one of the following types:', allowedtypes)
                sys.exit()
        
        
    
# Checks the fields of a WFDB record against one another to ensure there is no inconsistency
# Method belongs to WFDBrecord
def checkfieldcohesion(self, usesignals):
    
    # Potential fields to check: d_signals, nsig, siglen, filename, fmt, initvalue, checksum
    
    
    # Tricky part: just because nsig>0 doesn't mean you have to check d_signals. 
    
    # wait, if they say usesignals, but signals == none, or nsig = 0, then what? check previous functions like 
    # setsignalfeatures
    
    
    if usesignals: 
        # Match actual against stated signal shape
        if (self.siglen, self.nsig) != self.d_signals.shape:
            sys.exit('siglen and nsig do not match shape of d_signals: '+)
            
        # Make sure there are no values out of bounds for the digital format
        for ch in range(0, nsig):
            fmt = self.fmt[ch]
            dmin, dmax = digi_bounds(fmt)
            if (min(self.d_signals[:,ch])<dmin) or (max(self.d_signals[:,ch])>dmax):
                sys.exit("Channel "+str(ch)+" must only contain values between "+str(dmin)+" and "+str(dmax)+"for fmt = "+str(fmt))
                
    if self.nsig>0:
        # Check that each file has the same specified fmt and byteoffset
        datfmts = {}
        datoffsets = {}
        for ch in range(0,self.nsig):
            if self.filename[ch] not in datfmts:
                datfmts[self.filename[ch]] = self.fmt[ch]
            else:
                if datfmts[self.filename[ch]] != self.fmt[ch]:
                    sys.exit('Each filename (dat file) specified must have the same fmt')
                
            if self.filename[ch] not in datoffsets:
                datoffsets[self.filename[ch]] = self.byteoffset[ch]
            else:
                if datoffsets[self.filename[ch]] != self.byteoffset[ch]:
                    sys.exit('Each filename (dat file) specified must have the same byte offset')
                    
    
datformats = ["80","212","16","24","32"]
    
    
# Return min and max digital values for each format type.
def digi_bounds(fmt):
    if fmt == '80':
        return (-127, 127)
    elif fmt == '212':
        return (-2047, 2047)
    elif fmt == '16':
        return (-32767, 32767)
    elif fmt == '24':
        return (-8388607, 8388607)
    elif fmt == '32':
        return (-2147483647, 2147483648)
    
# Return nan value for the format type (accepts lists) 
def digi_nan(fmt):
    if type(fmt) == list:
        diginans = []
        for f in fmt:
            diginans.append(digi_nan(f))
        return diginans
        
    if fmt == '80':
        return -128
    elif fmt == '212':
        return -2048
    elif fmt == '16':
        return -32768
    elif fmt == '24':
        return -8388608
    elif fmt == '32':
        return -2147483648
    
# Returns the analogue to digital conversion for a WFDBrecord signal stored in p_signals. 
# The p_signals, fmt, gain, and baseline fields must all be valid.
def adc(self):
    
    # The digital nan values for each channel
    dnans = digi_nan(self.fmt) 
    
    d_signal = self.p_signals * self.gain + self.baseline
    
    for ch in range(0, np.shape(self.p_signals)[1]):
        # Nan values 
        nanlocs = np.isnan(self.p_signals[:,ch])
        if nanlocs.any():
            d_signal[nanlocs,ch] = dnans[ch]
    
    return d_signal

# Returns the digital to analogue conversion for a WFDBrecord signal stored in d_signals
# The d_signals, fmt, gain, and baseline fields must all be valid.
def dac(self):
    
    # The digital nan values for each channel
    dnans = digi_nan(fmt) 
    
    # Get nan indices, indicated by minimum value. 
    nanlocs = self.d_signals == dnans
    
    p_signal = (self.signals - self.baseline)/self.gain
        
    p_signal[nanlocs] = np.nan
            
    return p_signal
    
# Compute appropriate gain and baseline parameters given a physical signal and the fmts 
# Already considered for multiple formats. 
def adcparams(signals, fmt):
         
    # digital - baseline / gain = physical          !! gain CANNOT BE 0 !!
    # physical * gain + baseline = digital

    gains = []
    baselines = []
    
    # min and max ignoring nans, unless whole channel is nan. Should suppress error message. 
    minvals = np.nanmin(signals, axis=0) 
    maxvals = np.nanmax(signals, axis=0)
    
    dnans = digi_nan(fmt)
    
    for ch in range(0, np.shape(signals)[1]):
        dmin, dmax = digi_bounds(fmt[ch])
        dnan = dnans[ch]
        
        pmin = minvals[ch]
        pmax = maxvals[ch]
        
        # map values using full digital range.
        
        # If the entire signal is nan, just put any. 
        if pmin == np.nan:
            gain = 1 
            baseline = 1
        # If the signal is just one value, store all values as digital 1. 
        elif pmin == pmax:
            if minval ==0:
                gain = 1
                baseline = 1
            else:
                gain = 1/minval # wait.. what if minval is 0... 
                baselin = 0 
        else:
            
            gain = (dmax-dmin) / (pmax - pmin)
            baseline = dmin - gain * pmin

        # What about roundoff error? Make sure values don't map to beyond range. 
        baseline = np.floor(baseline) # baseline.astype('int64')
        
        # WFDB library limits...     
        if abs(gain)>214748364 or abs(baseline)>2147483648:
            sys.exit('Chen, please fix this')
                
        gains.append(gain)
        baselines.append(baseline)     
    
    return (gains, baselines)
    
          
# Set the field if there is a default. Otherwise do nothing and return.
# This method WILL overwrite fields already set. To avoid this, do not call it on fields already set.
# Method of WFDBbaserecord
def setdefault(self, field):
    
    # Note that some default fields rely on the values of other fields. This is ok because the dependent
    # fields are mandatory in all writing records, and are checked previously because they are ordered earlier
    # by getwritefields. 
    
    # This method does not calculate values based from either p_signals or d_signals.
    # That is done, if specified, in the 'setsignalfeatures' method. 
    
    # Record specification fields
    if field == 'basetime':
        self.basetime = '00:00:00'
        
    # Signal specification fields    
    elif field == 'filename':
        self.filename = self.nsig*[self.recordname+'.dat']
    elif field == 'adcres':
        self.adcres=wfdbfmtres(self.fmt)
    elif field == 'adczero':
        self.adczero = self.nsig*[0]
    elif field == 'blocksize':
        self.blocksize = self.nsig*[0]
        
    # To do for multirecord (or maybe it's better not in this function): elif field == 'seglen':
 

# Estimate the resolution of each signal in a multi-channel signal in bits. Maximum of 32 bits. 
reslevels = np.power(2, np.arange(0,33))
def estres(signals):
    # Estimate the number of steps as the range divided by the minimum increment. 
    
    nsig = np.size(signals)[1]
    # The estimated resolution in bits rounded up
    res = np.zeros(nsig)
    
    for ch in range(0, nsig):
        sortedsig = np.sort(signals[:,ch])
        min_inc = min(np.diff(sortedsig))
        
        if min_inc == 0:
            # Case where signal is flat. Resolution is 0.  
            continue
        else:
            nlevels = 1 + (sortedsig[-1]-sortedsig[0])/min_inc
            if nlevels>=reslevels[-1]:
                res[ch] = 32
            else:
                res[ch] = np.where(reslevels>nlevels)[0][0]
            
    return res
    

# Return the most suitable wfdb format to use given a signal resolution. Limited in output options. 
def wfdbfmt(res):
    if res<=8:
        return '80'
    elif res<=12:
        return '212'
    elif res<=16:
        return '16'
    elif res<=24:
        return '24'
    else:
        return '32'

# Return the resolution of the WFDB format. Returns a list if the input has multiple items. 
def wfdbfmtres(fmt):
    
    if type(fmt)==list:
        numel = fmt.shape[0]
        res = []
        for i in range(0, numel):
            res.append(wfdbfmtres(fmt[i]))
        return res
    
    if fmt in ['8', '80']:
        return 8
    elif fmt in ['310', '311']:
        return 10
    elif fmt == '212':
        return 12
    elif fmt in ['16', '61']:
        return 16
    elif fmt = '24':
        return 24
    elif fmt = '32':
        return 32
    else:
        sys.exit('Invalid WFDB format.')
    
# Calculate the checksums of a multi-channel digital signal
# Method of WFDBrecord. Class method. 
def calc_checksum(signals):
    return list(np.sum(signals, 0) % 65536)

    
# Retrieve a copy of the wfdb signals field, either digital or physical. 
# Returned array will NOT be a reference to the same object.
# Method of WFDBrecord
def getsignalscopy(self, physical = 1)
    
    if not self.signals:
        sys.exit('WFDBrecord has no signals')
    
    # Return the physical signals
    if physical:
        if self.p_signals != None:
            return self.psignals.copy()
        # Have to do dac
        elif self.d_dignals != None:
            return dac(self.d_signals)
        else:
            sys.exit('The WFDBrecord object contains neither p_signals nor d_signals')
    # Return the digital signals
    else:
        if self.d_signals != None:
            return self.dsignals.copy()
        # Have to do adc
        elif self.a_signals != None:
            return adc(self.a_signals)
        else:
            sys.exit('The WFDBrecord object contains neither p_signals nor d_signals')



SyntaxError: invalid syntax (<ipython-input-1-cdf254654092>, line 44)